
Final Capstone Project
Introduction
Alex wants to open a Pizza place in one of the neighborhoods in Washington, D.C. I am going to recommend him the best neighborhood to open this place.

Business Problem
We will have need information about the neighborhoods in Washington DC for Alex to open his Pizza Place. Additionally, we need to make sure that the neighborhood he opens the pizza place will have enough customers(we look look at university/ school/ offices data for this) and also make sure there are not a large amount of other pizza places in that neighborhood.

Data
To recommend Alex the best neighborhood to open a Pizza place, we will leverage the following sources for data

Neighborhoods in Washington, D.C: https://en.wikipedia.org/wiki/Neighborhoods_in_Washington,_D.C.
The coordinates of these neighborhoods from Open Street Map APIs
Extract following venues data from FourSquare API
Pizza Place in the neighborhoods
College & University
Offices

In [ ]:
!conda install -c anaconda beautifulsoup4 
!conda install -c anaconda requests
!conda install -c conda-forge geocoder 
!conda install -c conda-forge folium=0.5.0 --yes 
!conda install -c conda-forge geopy --yes

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: - ^C
failed

CondaError: KeyboardInterrupt

Solving environment: - 

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import geocoder # import geocoder
import folium 
from geopy.geocoders import Nominatim
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import re
print('Libraries Imported')


In [ ]:
website_url = requests.get('https://en.wikipedia.org/wiki/Neighborhoods_in_Washington,_D.C.').text
soup = BeautifulSoup(website_url,'lxml')
neighbourhoods = []
headers = soup.find_all('ul')
links =[]
for x in range(2,10):
    temp =[]
    temp = headers[x].find_all('li')
    for y in temp:
        links.append(y)
for x in links:
    a = x.find_all('a')
    if(len(a) > 0):
        temp = re.split(' \(|, ',((x.find('a')).get('title')))#append neighbourhoods in array for hyperlink
        if temp[0] not in neighbourhoods:
            neighbourhoods.append(temp[0])
    else:
        if x.text not in neighbourhoods:
            neighbourhoods.append(x.text) #append neighborhoods 
wash_dc_neighbourhoods=pd.DataFrame()
wash_dc_neighbourhoods['Neighbourhoods'] = neighbourhoods
wash_dc_neighbourhoods.head()

In [ ]:
print('There are {} neighbourhoods in Washington D.C.'.format(wash_dc_neighbourhoods.shape[0]))

In [ ]:
neighbourhoodlatitude =[]
neighbourhoodlongitude =[]
for index, row in wash_dc_neighbourhoods.iterrows():
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search.php?q=',
                             row['Neighbourhoods']+', Washington D.C.',
                             '&format=json&polygon=0')
    response = requests.get(url).json()
    if(len(response) == 0):
        indexNames = wash_dc_neighbourhoods[ wash_dc_neighbourhoods['Neighbourhoods'] == row['Neighbourhoods']].index
        wash_dc_neighbourhoods.drop(indexNames,inplace=True)
    else:
        response = response[0]
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
        neighbourhoodlatitude.append(output[0])
        neighbourhoodlongitude.append(output[1])
wash_dc_neighbourhoods['Latitude'] = neighbourhoodlatitude
wash_dc_neighbourhoods['Longitude'] = neighbourhoodlongitude
wash_dc_neighbourhoods.head()

In [ ]:
address = 'Washington, D.C.'

geolocator = Nominatim(user_agent="dc_explorer")
wash_dc_location = geolocator.geocode(address)
wash_dc_latitude = wash_dc_location.latitude
wash_dc_longitude = wash_dc_location.longitude
print('The geographical co-ordinates of Washington, D.C. are {}, {}.'.format(wash_dc_latitude, wash_dc_longitude))

In [ ]:
map_washington_dc = folium.Map(location=[wash_dc_latitude, wash_dc_longitude], zoom_start=12)

# add markers to map
for lat, lng, neighbourhood in zip(wash_dc_neighbourhoods['Latitude'], wash_dc_neighbourhoods['Longitude'], wash_dc_neighbourhoods['Neighbourhoods']):
    label = neighbourhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='navy',
        fill=True,
        fill_color='royalblue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_washington_dc)  
    
map_washington_dc

In [ ]:
CLIENT_ID = 'IQBZI3LY1CFPHDSLB04WSNB13J4BYM2SPVUH13OHMHLBWYXL' # your Foursquare ID
CLIENT_SECRET = 'SXGGJXB5HJSVRLZ5FLOOD3HXBHS2QFMTNHFS1CXBAZXDHB3N' # your Foursquare Secret
VERSION = '20190801' # Foursquare API version
LIMIT = 250 # limit of number of venues returned by Foursquare API

In [ ]:
import urllib
def getNearbyVenues(neighbourhood, latitudes, longitudes, categoryIds, radius):
    try:
        wash_dc_venues_list=[]
        for name, lat, lng in zip(neighbourhood, latitudes, longitudes):
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT, categoryIds)
            # make the GET request
            response = requests.get(url).json()
            results = response['response']['venues']
            # return only relevant information for each nearby venue
            for venues in results:
                success = False
                try:
                    category = venues['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    wash_dc_venues_list.append([(name, lat, lng, venues['name'], venues['location']['lat'], venues['location']['lng'],
                        venues['categories'][0]['name']
                    )])

        wash_dc_nearby_venues = pd.DataFrame([item for wash_dc_venue_list in wash_dc_venues_list for item in wash_dc_venue_list])
        wash_dc_nearby_venues.columns = ['Neighbourhoods', 'Latitude', 'Longitude', 'Venue_Name', 'Venue_Latitude', 'Venue_Longitude', 'Venue_Category']
    except:
        print(url)

    return(wash_dc_nearby_venues)

In [ ]:
wash_dc_venues_pizza = getNearbyVenues(neighbourhood=wash_dc_neighbourhoods['Neighbourhoods'], latitudes=wash_dc_neighbourhoods['Latitude'], longitudes=wash_dc_neighbourhoods['Longitude'], categoryIds='4bf58dd8d48988d1ca941735', radius=750)
wash_dc_venues_pizza.head()

In [ ]:
map_washington_dc_pizza_places = folium.Map(location=[wash_dc_latitude, wash_dc_longitude], zoom_start=12)

# add markers to map
for venue_latitude, venue_longitude, neighbourhood, venue_name in zip(wash_dc_venues_pizza['Venue_Latitude'], wash_dc_venues_pizza['Venue_Longitude'], wash_dc_venues_pizza['Neighbourhoods'], wash_dc_venues_pizza['Venue_Name']):
    label = '{} - {}'.format(venue_name, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [venue_latitude, venue_longitude],
        radius=5,
        popup=label,
        color='maroon',
        fill=True,
        fill_color='lightcoral',
        fill_opacity=0.7,
        parse_html=False).add_to(map_washington_dc_pizza_places)  
    
map_washington_dc_pizza_places

In [ ]:
wash_dc_venues_highschools = getNearbyVenues(neighbourhood=wash_dc_neighbourhoods['Neighbourhoods'], latitudes=wash_dc_neighbourhoods['Latitude'], longitudes=wash_dc_neighbourhoods['Longitude'], categoryIds='4bf58dd8d48988d13d941735',radius=750)
wash_dc_venues_highschools.head()

In [ ]:
map_washington_dc_highschools = folium.Map(location=[wash_dc_latitude, wash_dc_longitude], zoom_start=12)

# add markers to map
for venue_latitude, venue_longitude, neighbourhood, venue_name in zip(wash_dc_venues_highschools['Venue_Latitude'], wash_dc_venues_highschools['Venue_Longitude'], wash_dc_venues_highschools['Neighbourhoods'], wash_dc_venues_highschools['Venue_Name']):
    label = '{} - {}'.format(venue_name, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [venue_latitude, venue_longitude],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='honeydew',
        fill_opacity=0.7,
        parse_html=False).add_to(map_washington_dc_highschools)  
    
map_washington_dc_highschools

In [ ]:
wash_dc_venues_university = getNearbyVenues(neighbourhood=wash_dc_neighbourhoods['Neighbourhoods'], latitudes=wash_dc_neighbourhoods['Latitude'], longitudes=wash_dc_neighbourhoods['Longitude'], categoryIds='4bf58dd8d48988d1ae941735', radius=750)
wash_dc_venues_university.head()

In [ ]:
map_washington_dc_universities = folium.Map(location=[wash_dc_latitude, wash_dc_longitude], zoom_start=12)

# add markers to map
for venue_latitude, venue_longitude, neighbourhood, venue_name in zip(wash_dc_venues_university['Venue_Latitude'], wash_dc_venues_university['Venue_Longitude'], wash_dc_venues_university['Neighbourhoods'], wash_dc_venues_university['Venue_Name']):
    label = '{} - {}'.format(venue_name, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [venue_latitude, venue_longitude],
        radius=5,
        popup=label,
        color='rebeccapurple',
        fill=True,
        fill_color='plum',
        fill_opacity=0.7,
        parse_html=False).add_to(map_washington_dc_universities)  
    
map_washington_dc_universities

In [ ]:
def mergedf(wash_dc_neighbourhoods_venues, df, newcolname):
    grouped_wash_dc_venues = pd.DataFrame()
    grouped_wash_dc_venues=df.groupby('Neighbourhoods').count()
    for neighbourhoods in wash_dc_neighbourhoods_venues['Neighbourhoods']:
        if((grouped_wash_dc_venues.index).contains(neighbourhoods)):
            wash_dc_neighbourhoods_venues.loc[wash_dc_neighbourhoods_venues['Neighbourhoods'] == neighbourhoods,newcolname] = grouped_wash_dc_venues.loc[neighbourhoods, 'Venue_Name']
    return wash_dc_neighbourhoods_venues

In [ ]:
wash_dc_neighbourhoods_venues = pd.DataFrame()
wash_dc_neighbourhoods_venues = wash_dc_neighbourhoods.copy()
wash_dc_neighbourhoods_venues = mergedf(wash_dc_neighbourhoods_venues, wash_dc_venues_pizza, 'Pizza')
wash_dc_neighbourhoods_venues = mergedf(wash_dc_neighbourhoods_venues, wash_dc_venues_highschools, 'High_Schools')
wash_dc_neighbourhoods_venues = mergedf(wash_dc_neighbourhoods_venues, wash_dc_venues_university, 'Universities')
wash_dc_neighbourhoods_venues = mergedf(wash_dc_neighbourhoods_venues, wash_dc_venues_offices, 'Offices')
wash_dc_neighbourhoods_venues.fillna(value=0,inplace=True)
wash_dc_neighbourhoods_venues.head()

In [ ]:
weight_pizza = -1 # negative weight, because Alex wants to avoid concurrence as much as possible
weight_highschools = 1 #high school customers
weight_universities = 1.5 # university customers
weight_offices = 2 # office customers

In [ ]:
weighted_wash_dc_neighbourhoods_venues = wash_dc_neighbourhoods_venues[['Neighbourhoods']].copy()
weighted_wash_dc_neighbourhoods_venues['Score'] = wash_dc_neighbourhoods_venues['Pizza'] * weight_pizza + wash_dc_neighbourhoods_venues['High_Schools'] * weight_highschools + wash_dc_neighbourhoods_venues['Universities'] * weight_universities + wash_dc_neighbourhoods_venues['Offices'] * weight_offices
weighted_wash_dc_neighbourhoods_venues = weighted_wash_dc_neighbourhoods_venues.sort_values(by=['Score'], ascending=False)
weighted_wash_dc_neighbourhoods_venues

In [ ]:
map_dupontcircle_venues = folium.Map(location=[wash_dc_neighbourhoods[wash_dc_neighbourhoods['Neighbourhoods'] == 'Foggy Bottom']['Latitude'].values[0], wash_dc_neighbourhoods[wash_dc_neighbourhoods['Neighbourhoods'] == 'Dupont Circle']['Longitude'].values[0]], zoom_start=15)

# Plot pizza places in Dupont Circle
dupont_venues_pizza = wash_dc_venues_pizza[wash_dc_venues_pizza['Neighbourhoods'] == 'Foggy Bottom']
for venue_latitude, venue_longitude, neighbourhood, venue_name in zip(dupont_venues_pizza['Venue_Latitude'], dupont_venues_pizza['Venue_Longitude'], dupont_venues_pizza['Neighbourhoods'], dupont_venues_pizza['Venue_Name']):
    label = '{} - {}'.format(venue_name, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [venue_latitude, venue_longitude],
        radius=5,
        popup=label,
        color='maroon',
        fill=True,
        fill_color='lightcoral',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dupontcircle_venues) 

# Plot high schools in Dupont Circle
dupont_venues_highschools = wash_dc_venues_highschools[wash_dc_venues_highschools['Neighbourhoods'] == 'Foggy Bottom']    
for venue_latitude, venue_longitude, neighbourhood, venue_name in zip(dupont_venues_highschools['Venue_Latitude'], dupont_venues_highschools['Venue_Longitude'], dupont_venues_highschools['Neighbourhoods'], dupont_venues_highschools['Venue_Name']):
    label = '{} - {}'.format(venue_name, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [venue_latitude, venue_longitude],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='honeydew',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dupontcircle_venues)  

# Plot universities in Dupont Circle
dupont_venues_universties = wash_dc_venues_university[wash_dc_venues_university['Neighbourhoods'] == 'Foggy Bottom']
for venue_latitude, venue_longitude, neighbourhood, venue_name in zip(dupont_venues_universties['Venue_Latitude'], dupont_venues_universties['Venue_Longitude'], dupont_venues_universties['Neighbourhoods'], dupont_venues_universties['Venue_Name']):
    label = '{} - {}'.format(venue_name, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [venue_latitude, venue_longitude],
        radius=5,
        popup=label,
        color='rebeccapurple',
        fill=True,
        fill_color='plum',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dupontcircle_venues) 

# Plot offices in Dupont Circle
dupont_venues_offices = wash_dc_venues_offices[wash_dc_venues_offices['Neighbourhoods'] == 'Foggy Bottom']
for venue_latitude, venue_longitude, neighbourhood, venue_name in zip(dupont_venues_offices['Venue_Latitude'], dupont_venues_offices['Venue_Longitude'], dupont_venues_offices['Neighbourhoods'], dupont_venues_offices['Venue_Name']):
    label = '{} - {}'.format(venue_name, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [venue_latitude, venue_longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='silver',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dupontcircle_venues)  

map_dupontcircle_venues
